In [1]:
import pandas as pd
import numpy as np
import statistics as stats

In [2]:
data = pd.read_csv("../data/listings.csv")

In [3]:
# list of columns of interest
focusVariables = ["id", "accommodates", "bathrooms", "bedrooms", "beds", "price", "number_of_reviews", 
                  "review_scores_rating", "review_scores_cleanliness", "review_scores_checkin", 
                  "review_scores_communication", "review_scores_location", "review_scores_value"]

# Copy only comlumns to keep
cleanData = data[focusVariables].copy()

In [4]:
# Removes '$', ',', ' ', and replaces with '' (empty)
cleanData = cleanData.replace(["\$", ",", " "], "", regex=True)        

In [5]:
# Create dummy variables
amenities = data['amenities']
amenities = amenities.replace(["{", "}", "\""], "", regex=True).replace([" "], "", regex=True)
amenities = amenities.str.split(",")

dummies = pd.get_dummies(amenities.apply(pd.Series).stack()).sum(level=0)
dummies = dummies.drop(columns=[''])

In [6]:
# Join dummy columns to clean dataframe
cleanData = cleanData.join(dummies)

In [7]:
# Remove null & NaN values
cleanData = cleanData.dropna()

In [8]:
# Create copy of only data with 80 or higher review scores rating
cleanDataTarget = cleanData[cleanData.review_scores_rating >= 80]

In [9]:
# Export clean data to CSV
export_filename = '../data/clean_listings.csv'
cleanData.to_csv(export_filename, index=False)

# Export clean data with 80 or higher review scores rating
export_filename = '../data/target_clean_listings.csv'
cleanDataTarget.to_csv(export_filename, index=False)